In [213]:
import math

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Импортируем датасет

In [214]:
path = '/home/dmitry/reposit/Pet_project.test/_data.csv'

In [215]:
pet = pd.read_csv(path, index_col=0)

# Оставляем только объявления, которые относятся к городу Москва

In [216]:
pet['Адрес']  = pet['Адрес'].str.split(', ')
pet['Адрес'] = pet['Адрес'].apply(lambda x: x[0])

In [217]:
pet = pet[pet['Адрес'] =='Москва']
pet = pet.reset_index(drop=True)

# Создаем колонку - "price", выделив цену из колонки "цена"

In [218]:
lst = []
for item in pet['Цена'].tolist():
    if '$' in item:
        lst.append(95*float(item[:item.find('$')]))
    elif '€' in item:
        lst.append(102*float(item[:item.find('€')]))
    else:
        lst.append(float(item[:item.find('руб')]))

In [219]:
pet['price'] = lst

# Создаем колонку - "total_area", выделив общую площадь из колонки "Площадь, м2"

In [220]:
lst2 = []
for j in pet['Площадь, м2'].tolist():
    j1 = j.split('/')
    lst2.append(float(j1[0]))
pet['total_area'] = lst2

# Удаляем ненужные для дальнейшего анализа колонки

In [221]:
pet = pet.drop('Площадь, м2',axis=1)
pet = pet.drop('Тип', axis=1)
pet = pet.drop('Серия дома', axis=1)
pet = pet.drop('Название ЖК', axis=1)
pet = pet.drop('Телефоны', axis=1)
pet = pet.drop('Ссылка на объявление', axis=1)
pet = pet.drop('Цена', axis=1)
pet = pet.drop('Площадь комнат, м2',axis=1)
pet = pet.drop('Окна',axis=1)

# Из колонки "Количество комнат" выделяем численное значение и заполняем пустые строки средним значением

In [222]:
pet['Количество комнат'] = pet['Количество комнат'].str.replace(r'\D', '', regex=True)
pet['Количество комнат'] = pd.to_numeric(pet['Количество комнат'], errors='coerce')
pet['Количество комнат'] = pet['Количество комнат'].fillna(pet['Количество комнат'].mean()).round(1)

# Выделяем численное значения времени ходьбы до метро, меняем аномально большие значения и пропуски на "40 минут".

In [223]:
pet['Метро'] = pet['Метро'].str.replace(r'\D', '', regex=True)
pet['Метро'] = pd.to_numeric(pet['Метро'], errors='coerce')
pet['Метро'] = pet['Метро'].apply(lambda x: 40 if x > 40 else x)
pet['Метро'] = pet['Метро'].fillna(pet['Метро'].max())

# Заполняем пустые строки в колонке "Санузел" наиболее часто встречающимся значением, т.к. мы предположили, что это значение забыли указать.

In [224]:
mode_sanuzel = pet['Санузел'].mode().iloc[0]
pet['Санузел'] = pet['Санузел'].fillna(mode_sanuzel)

# Пустые строки в колонке "Высота потолков, м" были заменены на медианное значение, т.к. при анализе замечены аномальные значения.

In [225]:
pet['Высота потолков, м'] = pet['Высота потолков, м'].fillna(pet['Высота потолков, м'].median())

# Во всех оставшихся колонках, было принято решение - на место пропусков записать отсутствие информации/отсутствие опции

In [226]:
pet['Мусоропровод'] = pet['Мусоропровод'].fillna('Нет')
pet['Дополнительно'] = pet['Дополнительно'].fillna('Нет мебели')
pet['Лифт'] = pet['Лифт'].fillna('Нет лифта')
pet['Ремонт'] = pet['Ремонт'].fillna('Нет ремонта')
pet['Парковка'] = pet['Парковка'].fillna('Нет данных')
pet['Балкон'] = pet['Балкон'].fillna('Нет денных')
pet['Можно с детьми/животными'] = pet['Можно с детьми/животными'].fillna('Нет денных')

# Приводим названия колонок к общепринятому формату.

In [227]:
pet = pet.rename(columns={'ID  объявления': 'ad_id',
    'Количество комнат': 'number_of_rooms',
    'Метро': 'metro',
    'Адрес': 'address',
    'Дом': 'house',
    'Парковка': 'parking',
    'Описание': 'description',
    'Ремонт': 'renovation',
    'Балкон': 'balcony',
    'Санузел': 'bathroom',
    'Можно с детьми/животными': 'children_pets',
    'Дополнительно': 'additional_info',
    'Высота потолков, м': 'ceiling_height_(m)',
    'Лифт': 'elevator',
    'Мусоропровод': 'trash_chute',
    'price': 'price',
    'Общая площадь': 'total_area'})

# Сохраняем очищенный датасет.

In [229]:
pet.to_csv('data.csv', index=False) 